In [120]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
import preprocessing # https://github.com/Snowflake-Labs/snowpark-python-demos/tree/main/sp4py_utilities
from snowflake.snowpark.functions import col

import getpass
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
nj07294.ap-southeast-1

In [121]:
accountname = getpass.getpass() # ORGNAME-ACCOUNTNAME (separated by minus sign)

········


In [122]:
username = getpass.getpass()    # SNOWFLAKE-USERNAME

········


In [123]:
password = getpass.getpass()    # SNOWFLAKE-PASSWORD

········


In [124]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "FROSTBYTE_TASTY_BYTES",

    "warehouse": "HOL_WH"
}

session = Session.builder.configs(connection_parameters).create()

# Filter

In [125]:
sdf = session.table('ANALYTICS.ORDERS_V')
df=sdf.filter(col("COUNTRY")=='United States')

In [126]:
df=sdf.filter(col("COUNTRY")=='United States')

In [127]:
df.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"ORDER_ID"  |"TRUCK_ID"  |"ORDER_TS"           |"ORDER_DETAIL_ID"  |"LINE_NUMBER"  |"TRUCK_BRAND_NAME"  |"MENU_TYPE"  |"PRIMARY_CITY"  |"REGION"  |"COUNTRY"      |"FRANCHISE_FLAG"  |"FRANCHISE_ID"  |"FRANCHISEE_FIRST_NAME"  |"FRANCHISEE_LAST_NAME"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"E_MAIL"  |"PHONE_NUMBER"  |"CHILDREN_COUNT"  |"GENDER"  |"MARITAL_STATUS"  |"MENU_ITEM_ID"  |"MENU_ITEM_NAME"   

### Extracting weather data

In [128]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI",

    "warehouse": "HOL_WH"
}

wea_session = Session.builder.configs(connection_parameters).create()
wdf = wea_session.table('STANDARD_TILE.HISTORY_DAY')

In [129]:
wdf.show()


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [130]:
wdf2 = wdf.to_pandas()


In [131]:
wdf2.head()

,POSTAL_CODE,COUNTRY,DATE_VALID_STD,DOY_STD,MIN_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_AIR_2M_F,MAX_TEMPERATURE_AIR_2M_F,MIN_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,MAX_TEMPERATURE_WETBULB_2M_F,...,TOT_PRECIPITATION_IN,TOT_SNOWFALL_IN,TOT_SNOWDEPTH_IN,MIN_CLOUD_COVER_TOT_PCT,AVG_CLOUD_COVER_TOT_PCT,MAX_CLOUD_COVER_TOT_PCT,MIN_RADIATION_SOLAR_TOTAL_WPM2,AVG_RADIATION_SOLAR_TOTAL_WPM2,MAX_RADIATION_SOLAR_TOTAL_WPM2,TOT_RADIATION_SOLAR_TOTAL_WPM2
0,102103,NG,2021-07-18,199,73.8,77.4,85.4,72.0,73.0,74.3,...,0.00,0.0,0.0,54,92,100,0.0,167.5,830.9,4019.9
1,110054,IN,2021-07-18,199,79.3,84.1,91.1,76.6,78.0,79.6,...,0.38,0.0,0.0,24,80,100,0.0,96.4,373.6,2313.3
2,2044,AU,2021-07-18,199,49.4,55.0,61.9,45.9,48.9,52.5,...,0.00,0.0,0.0,0,3,8,0.0,143.9,559.5,3454.2
3,21745-690,BR,2021-07-18,199,62.0,68.5,78.0,61.1,64.9,68.8,...,0.03,0.0,0.0,9,44,99,0.0,164.9,638.3,3957.3
4,60596,DE,2021-07-18,199,55.5,69.1,80.6,54.9,63.5,69.6,...,0.00,0.0,0.0,0,4,56,0.0,323.3,867.4,7760.5


### Drop columns (like discount amount, etc)

In [132]:
df=df.drop(['ORDER_DISCOUNT_AMOUNT','ORDER_TAX_AMOUNT','ORDER_AMOUNT','PRICE','UNIT_PRICE','QUANTITY','MENU_ITEM_NAME','MENU_ITEM_ID','MARITAL_STATUS','GENDER','CHILDREN_COUNT','PHONE_NUMBER','E_MAIL','LAST_NAME','FIRST_NAME','CUSTOMER_ID','FRANCHISEE_FIRST_NAME','FRANCHISEE_LAST_NAME','FRANCHISE_ID','FRANCHISE_FLAG','LINE_NUMBER','ORDER_ID'])

### Transform order_ts to year, month, day, day of the week, hour, public holiday binary

In [133]:
df=df.withColumn("Month",F.month(df["ORDER_TS"]))
df=df.withColumn("DOW",F.dayofweek(df["ORDER_TS"]))
df=df.withColumn("Day",F.dayofmonth(df["ORDER_TS"]))
df=df.withColumn("Hour",F.hour(df["ORDER_TS"]))

# from datetime import datetime
# df['ORDER_TS'] = pd.to_datetime(df['ORDER_TS'])
# df['WOM'] = df['ORDER_TS'].dt.isocalendar().week #week of month

In [134]:
# Create public holiday column binary
public_holidays = [
    {'Month': 7, 'Day': 4, 'DOW': None, 'WOM': None},  # 4th of July
    {'Month': 12, 'Day': 24, 'DOW': None, 'WOM': None},  # Christmas Eve
    {'Month': 12, 'Day': 25, 'DOW': None, 'WOM': None},  # Christmas Day
    {'Month': 10, 'Day': None, 'DOW': 'Monday', 'WOM': 2},  # Columbus Day (second Monday in October)
    {'Month': 6, 'Day': 19, 'DOW': None, 'WOM': None},  # Juneteenth
    {'Month': 9, 'Day': None, 'DOW': 'Monday', 'WOM': 1},  # Labor Day (first Monday in September)
    {'Month': 1, 'Day': None, 'DOW': 'Monday', 'WOM': 3},  # Martin Luther King, Jr. Day (third Monday in January)
    {'Month': 5, 'Day': None, 'DOW': 'Monday', 'WOM': -1},  # Memorial Day (last Monday in May)
    {'Month': 1, 'Day': 1, 'DOW': None, 'WOM': None},  # New Year's Day
    {'Month': 12, 'Day': 31, 'DOW': None, 'WOM': None},  # New Year's Eve
    {'Month': 11, 'Day': None, 'DOW': 'Thursday', 'WOM': 4},  # Thanksgiving Day (fourth Thursday in November)
    {'Month': 11, 'Day': None, 'DOW': 'Wednesday', 'WOM': 4},  # Thanksgiving Eve (fourth Wednesday in November)
    {'Month': 2, 'Day': 14, 'DOW': None, 'WOM': None},  # Valentine's Day
    {'Month': 11, 'Day': 11, 'DOW': None, 'WOM': None},  # Veterans Day
    {'Month': 10, 'Day': 31, 'DOW': None, 'WOM': None},  # Halloween
    {'Month': 3, 'Day': 17, 'DOW': None, 'WOM': None},  # St. Patrick's Day
    {'Month': 11, 'Day': 25, 'DOW': 'Friday', 'WOM': None},  # Black Friday
    {'Month': 12, 'Day': 26, 'DOW': None, 'WOM': None},  # Boxing Day
]

In [135]:
def is_public_holiday(row):
    for holiday in public_holidays:
        if (holiday['Month'] is None or row['Month'] == holiday['Month']) and \
           (holiday['Day'] is None or row['Day'] == holiday['day']) and \
           (holiday['DOW'] is None or row['DOW'] == holiday['DOW']) and \
           (holiday['WOM'] is None or row['WOM'] == holiday['WOM']):
            
            return 1
    return 0

# Encoding M/F to 0/1
#df = df.with_column('public_hol', F.iff(is_public_holiday(F.col('CODE_GENDER')) == , 0, 1))

## Grouping

In [136]:
df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"ORDER_ID"  |"TRUCK_ID"  |"ORDER_TS"           |"ORDER_DETAIL_ID"  |"TRUCK_BRAND_NAME"  |"MENU_TYPE"  |"PRIMARY_CITY"  |"REGION"  |"COUNTRY"      |"LOCATION_ID"  |"ORDER_TOTAL"  |"MONTH"  |"DOW"  |"DAY"  |"HOUR"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2022-05-07  |451669196   |17          |2022-05-07 19:01:15  |883147685          |Smoky BBQ           |BBQ          |Denver          |Colorado  |United States  |5112           |46.0000        |5        |6      |7      |19      |
|2022-05-07  |451669196   |17          |2022-05-07 19:01:15  |883147686          |Sm

In [137]:
grouped_df = df.groupBy("DATE","TRUCK_ID","MONTH","HOUR","DOW","DAY","Menu_Type","LOCATION_ID" ).agg(F.sum("ORDER_TOTAL"))

# Joining with weather data

In [138]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI",

    "warehouse": "HOL_WH"
}

wea_session = Session.builder.configs(connection_parameters).create()
wdf = wea_session.table('STANDARD_TILE.HISTORY_DAY')

In [139]:
wdf=wea_session.sql("select * From STANDARD_TILE.HISTORY_DAY")

In [173]:
print(type(wdf))
print(type(semi_final_df))

<class 'snowflake.snowpark.dataframe.DataFrame'>
<class 'snowflake.snowpark.dataframe.DataFrame'>


In [141]:
grouped_df.show()

--------------------------------------------------------------------------------------------------------------------
|"DATE"      |"TRUCK_ID"  |"MONTH"  |"HOUR"  |"DOW"  |"DAY"  |"MENU_TYPE"     |"LOCATION_ID"  |"SUM(ORDER_TOTAL)"  |
--------------------------------------------------------------------------------------------------------------------
|2021-10-31  |30          |10       |8       |0      |31     |Sandwiches      |14929          |4828.0000           |
|2021-11-01  |19          |11       |11      |1      |1      |Mac & Cheese    |14943          |3674.0000           |
|2021-11-01  |18          |11       |12      |1      |1      |Tacos           |14851          |7927.5000           |
|2021-11-01  |18          |11       |18      |1      |1      |Tacos           |14774          |11326.0000          |
|2021-11-01  |20          |11       |11      |1      |1      |Ramen           |14951          |3287.7500           |
|2021-11-01  |20          |11       |16      |1      |1      |Ra

In [142]:
wdf_re=wdf.with_column_renamed(col("DATE_VALID_STD"), "DATE")

In [143]:
sdf_loc = session.table('RAW_POS.Location_New')

In [144]:
sdf_loc_dr=sdf_loc.drop("PLACEKEY","ISO_COUNTRY_CODE")

In [145]:
sdf_loc_dr.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------
|"LOCATION_ID"  |"LOCATION"                                        |"CITY"  |"REGION"  |"COUNTRY"      |"LAT"               |"LONG"               |"ZIPCODE"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------
|1030           |University Of Colorado Museum Of Natural History  |Denver  |CO        |United States  |40.00768674326263   |-105.26970066129032  |80802      |
|1031           |Denver Technological Center                       |Denver  |CO        |United States  |39.62735682905758   |-104.91269066527825  |80237      |
|1032           |Heritage Club At Denver Tech Center               |Denver  |CO        |United States  |39.62526657316754   |-104.91238594440485  |80237      |
|1033           |Porter Wound Care Cente

In [149]:
semi_final_df=grouped_df.join(sdf_loc_dr, grouped_df["LOCATION_ID"] == sdf_loc_dr["LOCATION_ID"])

In [150]:
semi_final_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"TRUCK_ID"  |"MONTH"  |"HOUR"  |"DOW"  |"DAY"  |"MENU_TYPE"     |"l_myea_LOCATION_ID"  |"SUM(ORDER_TOTAL)"  |"r_69f8_LOCATION_ID"  |"LOCATION"              |"CITY"  |"REGION"  |"COUNTRY"      |"LAT"               |"LONG"               |"ZIPCODE"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2021-10-31  |30          |10       |8       |0      |31     |Sandwiches      |14929                 |4828.0000           |14929                 |Capitol Hill Reservoir  |Denver  |CO        |United States  |39.7

In [109]:
wdf_re.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [151]:
semi_final_df=semi_final_df.with_column_renamed(col("ZIPCODE"), "POSTAL_CODE")

In [166]:
type(wdf_re["DATE"])

snowflake.snowpark.column.Column

In [168]:
final_df = semi_final_df.join(wdf_re, (semi_final_df["DATE"] == wdf_re["DATE"]))

In [170]:
weather=wdf_re.to_pandas()

In [172]:
 syntax=pd.io.sql.get_schema(weather, "weather_data")

In [169]:
final_df.show()

Failed to execute query [queryID: 01ac4c02-3200-bb18-0003-d8760005fa26]  SELECT  *  FROM ( SELECT  *  FROM (( SELECT "DATE" AS "l_rqp3_DATE", "TRUCK_ID" AS "TRUCK_ID", "MONTH" AS "MONTH", "HOUR" AS "HOUR", "DOW" AS "DOW", "DAY" AS "DAY", "MENU_TYPE" AS "MENU_TYPE", "l_myea_LOCATION_ID" AS "l_myea_LOCATION_ID", "SUM(ORDER_TOTAL)" AS "SUM(ORDER_TOTAL)", "r_69f8_LOCATION_ID" AS "r_69f8_LOCATION_ID", "LOCATION" AS "LOCATION", "CITY" AS "CITY", "REGION" AS "REGION", "COUNTRY" AS "l_rqp3_COUNTRY", "LAT" AS "LAT", "LONG" AS "LONG", "POSTAL_CODE" AS "l_rqp3_POSTAL_CODE" FROM ( SELECT "DATE", "TRUCK_ID", "MONTH", "HOUR", "DOW", "DAY", "MENU_TYPE", "l_myea_LOCATION_ID", "SUM(ORDER_TOTAL)", "r_69f8_LOCATION_ID", "LOCATION", "CITY", "REGION", "COUNTRY", "LAT", "LONG", "ZIPCODE" AS "POSTAL_CODE" FROM ( SELECT  *  FROM (( SELECT "DATE" AS "DATE", "TRUCK_ID" AS "TRUCK_ID", "MONTH" AS "MONTH", "HOUR" AS "HOUR", "DOW" AS "DOW", "DAY" AS "DAY", "MENU_TYPE" AS "MENU_TYPE", "LOCATION_ID" AS "l_myea_LOCATI

SnowparkSQLException: (1304): 002003 (02000): SQL compilation error:
Schema 'FROSTBYTE_TASTY_BYTES.STANDARD_TILE' does not exist or not authorized.